# PowerShell Abuse

## Overview

**Task.** Query for PowerShell abuse using PowerShell.  
**Purpose.** PowerShell is a command-line and scripting interface within the Windows operating system. Adversaries abuse PowerShell to download and execute malicious code.  
**Conditions.** PowerShell Remoting and Script Block Logging is enabled.  
**Standard.** You were able to perform a query and determine if suspicious or malicious activity has occurred.

## Setup

In [ ]:
$ComputerName = "localhost"
$FilterHashTable = @{
    LogName = "Microsoft-Windows-PowerShell/Operational"
    Id = 4104
}

filter Read-WinEvent {
    $WinEvent = [ordered]@{} 
    $XmlData = [xml]$_.ToXml()
    $SystemData = $XmlData.Event.System
    $SystemData | 
    Get-Member -MemberType Properties | 
    Select-Object -ExpandProperty Name |
    ForEach-Object {
        $Field = $_
        if ($SystemData[$Field].'#text') {
            $WinEvent.$Field = $SystemData[$Field].'#text'
        } else {
            $SystemData[$Field]  | 
            Get-Member -MemberType Properties | 
            Select-Object -ExpandProperty Name |
            ForEach-Object { 
                $WinEvent.$Field = @{}
                $WinEvent.$Field.$_ = $SystemData[$Field].$_
            }
        }
    }
    $XmlData.Event.EventData.Data |
    ForEach-Object { 
        $WinEvent.$($_.Name) = $_.'#text'
    }
    return New-Object -TypeName PSObject -Property $WinEvent
}

## Query & Analysis

In [ ]:
Get-WinEvent -ComputerName $ComputerName -FilterHashTable $FilterHashTable -MaxEvents 10 |
Read-WinEvent |
Select-Object -Property `
@{ Name = "TimeCreated"; Expression = { $_.TimeCreated.SystemTime }},
@{ Name = "SecurityId"; Expression = { $_.Security.UserId }},
ScriptBlockText 